In [12]:
!nvidia-smi

Sat Jan 24 19:14:05 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.05              Driver Version: 580.95.05      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        Off |   00000000:02:00.0 Off |                  N/A |
|  0%   33C    P1             64W /  575W |   32049MiB /  32607MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
!kill -9 213939

: 

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
from Generator.model import Diffusion

torch.cuda.empty_cache()
torch.cuda.synchronize()

model = Diffusion().to('cuda').eval()
dummy = torch.randn(8, 3, 128, 128, device='cuda')

with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.bfloat16):
    pred = model(dummy)

print(pred.shape)


OutOfMemoryError: CUDA out of memory. Tried to allocate 114.00 MiB. GPU 0 has a total capacity of 31.36 GiB of which 60.31 MiB is free. Including non-PyTorch memory, this process has 31.28 GiB memory in use. Of the allocated memory 30.68 GiB is allocated by PyTorch, and 11.72 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
t = torch.randint(0, 1000, (1,))

NameError: name 'torch' is not defined

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as T
import numpy as np
from PIL import Image
from tqdm import tqdm
from torch.utils.data import  DataLoader,random_split
from Generator.Dataset import CustomDataset
from Generator.NN import NNDenoiser
from Generator.add_noise import add_noise
from Generator.positional_encoding import positional_encoding

In [2]:
import os
os.chdir('../')

In [20]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None, add_noise=None, K=1000):
        self.root_dir = root_dir
        self.transform = transform
        self.add_noise = add_noise
        self.K = K

        self.images = [
            os.path.join(root_dir, f)
            for f in os.listdir(root_dir)
            if f.lower().endswith((".png", ".jpg", ".jpeg"))
        ]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.open(self.images[idx]).convert("RGB")

        # (C, H, W)
        if self.transform:
            img = self.transform(img)

        # Repeat image K times -> (K, C, H, W)
        imgs = img.unsqueeze(0).repeat(self.K, 1, 1, 1)

        # Noise (K, C, H, W)
        noise = torch.randn_like(imgs)

        # t in [0, 1) for each of the K diffusions (K, 1, 1, 1)
        t = torch.rand(self.K, 1, 1, 1)

        # xt = noisy images
        xt = self.add_noise(imgs, t, noise)

        return xt, t, noise


In [22]:
transform = T.Compose([
    T.Resize(128),
    T.CenterCrop(128),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3) 
])
dataset = CustomDataset("data/", transform=transform, add_noise=add_noise)

In [27]:
xt, t, noise = dataset[0]

In [29]:
xt[0].shape

torch.Size([3, 128, 128])

In [31]:
t[0].shape

torch.Size([1, 1, 1])

In [33]:
noise[0].shape

torch.Size([3, 128, 128])

In [25]:
len(dataset)

17534

In [36]:
class CustomNoiseImageDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)*1000

    def __getitem__(self, idx):
        img, t, noise  = self.dataset[idx % len(self.dataset)]
        img = img[idx]
        t = t[idx]
        noise = noise[idx]
        return img,t,noise

In [37]:
noise_dataset = CustomNoiseImageDataset(dataset)

In [38]:
len(noise_dataset)

17534000